In [ ]:
# Importando os pacotes 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import colorsys
import sklearn
plt.style.use('seaborn-talk')
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#Carregando o dataset
df = pd.read_csv('Safra_2018-2019.csv')  

In [ ]:
# Qual é a distribuição de dano na plantação por semana utilizando agrotóxicos 
# A grande maioria da plantação não apresenta danos

# Definindo a quantidade
labels = df.dano_na_plantacao.value_counts().index
num = len(df.Semanas_Utilizando.value_counts().index)

# Criando a lista de cores
colors = ['OliveDrab', 'Orange', 'OrangeRed', 'DarkCyan', 'Salmon', 'Sienna', 'Maroon', 'LightSlateGrey', 'DimGray']

# Gráfico de Pizza
fatias, texto = plt.pie(df.dano_na_plantacao.value_counts(), colors = colors, startangle = 90)
plt.axes().set_aspect('equal', 'datalim')
plt.legend(fatias, labels, bbox_to_anchor = (1.05,1))
plt.title("Dano na plantação por utilização de agrotóxico")
plt.show()
print("")

#Separação de dano nas plantações
#Verificando como os dados estão distribuidos

print("Sem danos: {0} ({1:0.2f}%)".format(len(df.loc[df['dano_na_plantacao'] == 0]), 
                                               (len(df.loc[df['dano_na_plantacao'] ==0])/len(df.index) * 100)))

print("Danos causados por outros motivos : {0} ({1:0.2f}%)".format(len(df.loc[df['dano_na_plantacao'] == 1]), 
                                               (len(df.loc[df['dano_na_plantacao'] == 1])/len(df.index) * 100)))

print("Danos gerados pelos pesticidas : {0} ({1:0.2f}%)".format(len(df.loc[df['dano_na_plantacao'] == 2]), 
                                               (len(df.loc[df['dano_na_plantacao'] == 2])/len(df.index) * 100)))

# Identificando a relação entre váriaveis atráves de gráfico:

In [ ]:
def plot_corr(df, size = 10):
    corr = df.corr()
    fig, ax = plt.subplots(figsize = (size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)),corr.columns)
    plt.yticks(range(len(corr.columns)),corr.columns)
    

In [ ]:
plot_corr(df)

In [ ]:
df.corr()

# Iniciando o split

In [ ]:
#Importando pacote de split
from sklearn.model_selection import train_test_split

In [ ]:
#Seleção de váriaveis preditoras [Feature Selection]
atributos = ['Estimativa_de_Insetos','Tipo_de_Cultivo','Tipo_de_Solo','Categoria_Pesticida', 'Doses_Semana', 'Semanas_Utilizando', 'Semanas_Sem_Uso', 'Temporada']

In [ ]:
#A váriavel a ser prevista
atributos_prev = ['dano_na_plantacao']

In [ ]:
#Criando objetos
x = df[atributos].values
y = df[atributos_prev].values

In [ ]:
split_teste_size = 0.30

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = split_teste_size, random_state = 42)

In [ ]:
#Tratamento de dados missing
from sklearn.preprocessing import Imputer

In [ ]:
preenche_0 = Imputer(missing_values=0, strategy= "mean", axis= 0)
#Substituir os valores de treino e teste
X_treino =  preenche_0.fit_transform(x_treino)
X_teste =  preenche_0.fit_transform(x_teste)

In [ ]:
#Importando métricas de exatidão dos modelos
from sklearn import metrics
#Importando pacote de modelo preditivo
from sklearn.linear_model import LogisticRegression

In [ ]:
modelo_v1 = LogisticRegression(C = 0.7, random_state=42)
modelo_v1.fit(X_treino, y_treino.ravel())

# Acuracia de teste

In [ ]:
nb_predict_test = modelo_v1.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_teste, nb_predict_test)))
print()

Salvando modelo para utilizar

In [ ]:
import pickle
filename = 'modelo_treinado_Scicrop.sav'
pickle.dump(modelo_v1, open(filename,'wb'))

Carregando modelo para utilizar com o conjunto de dados de 2020

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

# Carregando conjunto de dados de 2020

In [ ]:
df_2020 = pd.read_csv('Safra_2020.csv')
dano_na_plantacao = []
for line in df_2020.Temporada:
    dano_na_plantacao = 0
df_2020['dano_na_plantacao'] = dano_na_plantacao

In [ ]:
x_2020 = df_2020[atributos].values
y_2020 = df_2020[atributos_prev].values
#Retirando valores missing
preenche_0 = Imputer(missing_values=0, strategy= "mean", axis= 0)
x_2020 =  preenche_0.fit_transform(x_2020)
nb_predict_2020 = loaded_model.predict(x_2020)
y_2020 = nb_predict_2020
df_2020.drop(['dano_na_plantacao'],axis=1)
#Adicionando coluna preditiva
dano_na_plantacao = []
for line in df_2020.Temporada:
    dano_na_plantacao = nb_predict_2020
df_2020['dano_na_plantacao'] = dano_na_plantacao

# Separando dano nas plantações

In [ ]:
# Definindo a quantidade
labels = df_2020.dano_na_plantacao.value_counts().index
num = len(df_2020.Semanas_Utilizando.value_counts().index)

# Criando a lista de cores
colors = ['OliveDrab', 'Orange', 'OrangeRed', 'DarkCyan', 'Salmon', 'Sienna', 'Maroon', 'LightSlateGrey', 'DimGray']

# Gráfico de Pizza
fatias, texto = plt.pie(df_2020.dano_na_plantacao.value_counts(), colors = colors, startangle = 90)
plt.axes().set_aspect('equal', 'datalim')
plt.legend(fatias, labels, bbox_to_anchor = (1.05,1))
plt.title("Dano na plantação por utilização de agrotóxico")
plt.show()
print("")

#Separação de dano nas plantações
#Verificando como os dados ficaram distribuidos

print("Sem danos: {0} ({1:0.2f}%)".format(len(df_2020.loc[df_2020['dano_na_plantacao'] == 0]), 
                                               (len(df_2020.loc[df_2020['dano_na_plantacao'] ==0])/len(df.index) * 100)))

print("Danos causados por outros motivos : {0} ({1:0.2f}%)".format(len(df_2020.loc[df_2020['dano_na_plantacao'] == 1]), 
                                               (len(df_2020.loc[df_2020['dano_na_plantacao'] == 1])/len(df.index) * 100)))

print("Danos gerados pelos pesticidas : {0} ({1:0.2f}%)".format(len(df_2020.loc[df_2020['dano_na_plantacao'] == 2]), 
                                               (len(df_2020.loc[df_2020['dano_na_plantacao'] == 2])/len(df.index) * 100)))

In [ ]:
#salvando o modelo atualizado
df_2020.to_csv('Safra_2020-atualizada.csv')

# FIM !